In [1]:
#Import libraries
import pandas as pd
import statsmodels.api as sm

In [2]:
#Sample time series data
data = pd.Series([10,12,15,18, 22, 25, 28,30,32,35,10,12,15,18, 22, 25, 28,30,32,35,20,43,56,34],
                index=pd.date_range(start='2023-01-01', periods=24, freq='M'))

In [3]:
data

2023-01-31    10
2023-02-28    12
2023-03-31    15
2023-04-30    18
2023-05-31    22
2023-06-30    25
2023-07-31    28
2023-08-31    30
2023-09-30    32
2023-10-31    35
2023-11-30    10
2023-12-31    12
2024-01-31    15
2024-02-29    18
2024-03-31    22
2024-04-30    25
2024-05-31    28
2024-06-30    30
2024-07-31    32
2024-08-31    35
2024-09-30    20
2024-10-31    43
2024-11-30    56
2024-12-31    34
Freq: M, dtype: int64

In [4]:
#Creating lag features
def create_lag_feature(data, lags):
    lag_features = pd.DataFrame()
    for i in range(1, + lags + 1):
        lag_features[f'Lag_{i}'] = data.shift(i)
    return lag_features

In [5]:
#Calculating moving averages
def create_ma_features(data, window):
    ma_features = pd.DataFrame()
    for i in range(1, window+1):
        ma_features[f'MA_{i}'] = data.rolling(window=i).mean()
    return ma_features

In [6]:
#Calculate trend feature
def create_trend_feaure(data):
    trend_feat = pd.DataFrame()
    trend_feat['Trend'] = range(1, len(data)+1)
    trend_feat.set_index(data.index, inplace=True)
    return trend_feat

In [7]:
#Calculate seasonaliy feature
def create_seasonality_feaure(data, period):
    seasonality = pd.DataFrame()
    seasonality['Seasonaliy'] = data.diff(periods = period)
    return seasonality

In [8]:
#Create autocorrelation features
def create_autocorrelation_features(data, lags):
    auto_corr = pd.DataFrame()
    for i in range(1, lags+1):
        auto_corr[f'Autocorr_{i}'] = data.autocorr(lag=i)
    return auto_corr

In [9]:
#Function to calculate exponential Moving average
def create_ema(data, span):
    ema_feature = pd.DataFrame()
    ema_feature['EMA'] = data.ewm(span=span, adjust=False).mean()
    return ema_feature

In [10]:
#Function to caalculate cum sum
def create_cumulative_sum_feature(data):
    cumsum_feat = pd.DataFrame()
    cumsum_feat['Cumulative_Sum_3'] = data.rolling(window=3).mean()
    cumsum_feat['Cumulative_Sum_6'] = data.rolling(window=6).mean()
    return cumsum_feat

In [12]:
#Calculate trend feature
def create_trend_feaure1(data):
    data = pd.DataFrame(data)
    trend_feat = pd.DataFrame()
    result = sm.tsa.seasonal_decompose(data[0], model='additive')
    trend_feat['Trend 1'] = result.trend
    trend_feat.set_index(data.index, inplace=True)
    return trend_feat

#Calculate Seas feature
def create_seas_feaure1(data):
    data = pd.DataFrame(data)
    seas_feat = pd.DataFrame()
    result = sm.tsa.seasonal_decompose(data[0], model='additive')
    seas_feat['Seasonality 1'] = result.seasonal
    seas_feat.set_index(data.index, inplace=True)
    return seas_feat

In [13]:
#Lets call all the functions above and append the features to the original df
lag_features = create_lag_feature(data, 3)
ma_features = create_ma_features(data, 3)
trend_feature = create_trend_feaure(data)
seas_feature = create_seasonality_feaure(data, 1)
auto_corr = create_autocorrelation_features(data, 3)
ema_feat = create_ema(data, 3)
cum_sum = create_cumulative_sum_feature(data)

trend1 = create_trend_feaure1(data)
seas1 = create_seas_feaure1(data)

In [14]:
features = [lag_features, ma_features, trend_feature, seas_feature, auto_corr, ema_feat, cum_sum, trend1, seas1]
data = pd.concat([data]+features, axis=1)

In [15]:
data.head()

,0,Lag_1,Lag_2,Lag_3,MA_1,MA_2,MA_3,Trend,Seasonaliy,Autocorr_1,Autocorr_2,Autocorr_3,EMA,Cumulative_Sum_3,Cumulative_Sum_6,Trend 1,Seasonality 1
2023-01-31 00:00:00,10,NaN,NaN,NaN,10.0,NaN,NaN,1,NaN,NaN,NaN,NaN,10.00,NaN,NaN,NaN,-8.954861
2023-02-28 00:00:00,12,10.0,NaN,NaN,12.0,11.0,NaN,2,2.0,NaN,NaN,NaN,11.00,NaN,NaN,NaN,-6.329861
2023-03-31 00:00:00,15,12.0,10.0,NaN,15.0,13.5,12.333333,3,3.0,NaN,NaN,NaN,13.00,12.333333,NaN,NaN,-2.038194
2023-04-30 00:00:00,18,15.0,12.0,10.0,18.0,16.5,15.000000,4,3.0,NaN,NaN,NaN,15.50,15.000000,NaN,NaN,1.128472
2023-05-31 00:00:00,22,18.0,15.0,12.0,22.0,20.0,18.333333,5,4.0,NaN,NaN,NaN,18.75,18.333333,NaN,NaN,1.878472
